<a href="https://colab.research.google.com/github/Thanwarin/robot-webots/blob/tmp/emotion_detection_Predict_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We use the following library versions for this project:

- TensorFlow 2.19.0
- NumPy 2.0.2
- MediaPipe 0.10.14

These specific versions are required because the emotion detection ML model was trained
using TensorFlow 2.19.0 and these library versions. Using the exact versions ensures
compatibility and reproducibility of model inference results.


In [ ]:
# ==============================
# Install specific versions
# ==============================
!pip install tensorflow==2.19.0 numpy==2.0.2 mediapipe==0.10.14 --quiet

In [ ]:
# ==============================
# Imports
# ==============================
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import pandas as pd

print(f"tf: {tf.__version__}, np: {np.__version__}")

# ==============================
# Config
# ==============================
IMG_SIZE = 96
NUM_CLASSES = 7
emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

# ==============================
# Preprocess function
# ==============================
def preprocess_image(pixel_sequence):
    """
    Convert pixel string (FER2013) or 2D grayscale image to RGB,
    resize to IMG_SIZE, and normalize [0,1].
    """
    if isinstance(pixel_sequence, str):
        arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])
        if arr.size != 48*48:
            return None
        img_gray = arr.reshape(48,48).astype('float32')
    else:
        # if input is already a 2D grayscale image
        img_gray = pixel_sequence.astype('float32')
    img_resized = cv2.resize(img_gray, (IMG_SIZE, IMG_SIZE))
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
    return img_rgb / 255.0

def preprocess_dataframe(df, pixel_col='pixels'):
    """
    Convert dataframe to X array of images.
    Returns: X (numpy array)
    """
    X = []
    for _, row in df.iterrows():
        img = preprocess_image(row[pixel_col])
        if img is not None:
            X.append(img)
    return np.array(X, dtype='float32')

# ==============================
# Load Model
# ==============================
def load_model_from_path(path):
    """
    Load saved .h5 model from local path or Google Drive.
    """
    model = load_model(path)
    return model

# ==============================
# Prediction function
# ==============================
def predict_emotions(model, X, labels=emotion_labels):
    """
    Predict emotions for a batch of images.

    Args:
        model : trained Keras model
        X : numpy array of shape (N, IMG_SIZE, IMG_SIZE, 3)
        labels : list of emotion labels

    Returns:
        results : list of dicts
            'percentages' : dict of emotion -> probability*100
            'predicted_emotion' : emotion with max probability
            'max_probability' : float of max probability
    """
    preds_prob = model.predict(X, verbose=0)
    results = []
    for prob in preds_prob:
        percentages = {labels[i]: float(prob[i]*100) for i in range(len(labels))}
        max_idx = np.argmax(prob)
        results.append({
            'percentages': percentages,
            'predicted_emotion': labels[max_idx],
            'max_probability': float(prob[max_idx]*100)
        })
    return results

# ==============================
# Example Usage
# ==============================
# 1. Load your model
model_path = '/content/emotion_model_ver4.h5'  # change path
emotion_model = load_model_from_path(model_path)
print(emotion_model.summary())

# 2. Preprocess input images (from DataFrame)
# Example with FER2013 CSV
data = pd.read_csv('fer2013.csv')
# Take a small sample for demonstration
sample_df = data.sample(5, random_state=42)
X_input = preprocess_dataframe(sample_df)

# 3. Predict emotions
predictions = predict_emotions(emotion_model, X_input)

# 4. Show results
for i, pred in enumerate(predictions):
    print(f"Sample {i+1}:")
    print("Predicted Emotion:", pred['predicted_emotion'])
    print("Max Probability: {:.2f}%".format(pred['max_probability']))
    print("All Percentages:", {k: f"{v:.2f}%" for k,v in pred['percentages'].items()})
    print("-"*50)
